# TensorFlow Testing Field

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm #create process bar
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Load the dataset
data = pd.read_csv("mimic_mean_final.csv")

In [ ]:
# Filtering rows at the specific time zone
data = data[data['Time_Zone'] == 1]

In [ ]:
display(data)

In [ ]:
# Separate numeric and categorical columns
numeric_data = data.select_dtypes(include=np.number)
categorical_data = data.select_dtypes(exclude=np.number)

# Initialize the IterativeImputer
imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=0)

# Define the number of iterations for imputation
num_iterations = 10

# Calculate the total number of imputation steps
total_steps = num_iterations * numeric_data.shape[1]

# Create a progress bar for the imputation process
pbar = tqdm(total=total_steps)

# Perform imputation on the entire numeric dataset
for _ in range(num_iterations):
    imputer.fit(numeric_data)
    imputed_numeric_data = imputer.transform(numeric_data)
    pbar.update(numeric_data.shape[1])  # Update progress bar for each column imputed

# Close the progress bar
pbar.close()

# Convert imputed numeric data to a DataFrame
imputed_numeric_df = pd.DataFrame(imputed_numeric_data, columns=numeric_data.columns)

# Concatenate imputed numeric data with categorical data
imputed_data = pd.concat([categorical_data.reset_index(drop=True), imputed_numeric_df.reset_index(drop=True)], axis=1)

In [ ]:
display(imputed_numeric_df)

In [ ]:
# Export the merged DataFrame to a CSV file
imputed_full_df.to_csv('CSV\export\imputed_mimic_time_zone_1.csv', index=False)